In [49]:
import os
import shutil
import random
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
from PIL import Image
import torch.nn.functional as F
from sklearn.metrics import classification_report

In [34]:
# === PARAMETERS ===
root_folder = 'aug_processed_data'  # Folder with 'healthy' and 'unhealthy' subfolders
base_dir = 'split_data_EfficientNetB0'                        # New folder where train/val split will go
train_ratio = 0.8                              # 80% train, 20% val split


In [45]:
dataset = datasets.ImageFolder('aug_processed_data', transform=transform)

In [35]:
# === 1. Create train/val folders with same subfolder structure ===
def create_train_val_split(root_folder, base_dir, train_ratio=0.8):
    if os.path.exists(base_dir):
        shutil.rmtree(base_dir)
    os.makedirs(base_dir)

    train_dir = os.path.join(base_dir, 'train')
    val_dir = os.path.join(base_dir, 'val')
    os.makedirs(train_dir)
    os.makedirs(val_dir)

    classes = [d for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]

    for cls in classes:
        os.makedirs(os.path.join(train_dir, cls))
        os.makedirs(os.path.join(val_dir, cls))

        images = os.listdir(os.path.join(root_folder, cls))
        random.shuffle(images)

        train_count = int(len(images) * train_ratio)
        train_imgs = images[:train_count]
        val_imgs = images[train_count:]

        for img in train_imgs:
            shutil.copy(os.path.join(root_folder, cls, img), os.path.join(train_dir, cls, img))
        for img in val_imgs:
            shutil.copy(os.path.join(root_folder, cls, img), os.path.join(val_dir, cls, img))

    print(f"✅ Dataset split done! Train and val folders created at '{base_dir}'")


In [36]:
# === Run the split ===
create_train_val_split(root_folder, base_dir, train_ratio)

✅ Dataset split done! Train and val folders created at 'split_data_EfficientNetB0'


In [37]:
# === 2. Set up EfficientNet-B0 with Transfer Learning ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [38]:
# Use EfficientNet's recommended weights and transforms
weights = EfficientNet_B0_Weights.DEFAULT
transform = weights.transforms()

train_dataset = datasets.ImageFolder(os.path.join(base_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(base_dir, 'val'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [39]:
# Load model and modify classifier
model_effb0 = efficientnet_b0(weights=weights)
for param in model_effb0.parameters():
    param.requires_grad = False  # freeze base

num_features = model_effb0.classifier[1].in_features
model_effb0.classifier[1] = nn.Linear(num_features, len(train_dataset.classes))  # replace final layer
model_effb0 = model_effb0.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_effb0.classifier[1].parameters(), lr=0.001)

In [40]:
# === 3. Training loop ===
def train_model(epochs=10):
    best_val_acc = 0.0
    for epoch in range(epochs):
        model_effb0.train()
        total_loss, correct, total = 0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model_effb0(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        avg_loss = total_loss / len(train_loader)

        model_effb0.eval()
        val_correct, val_total, val_loss = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model_effb0(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)
        if val_acc > best_val_acc:
            best_val_acc = val_acc

        print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    print(f"\n🏆 Best Val Accuracy: {best_val_acc:.2f}%")

In [41]:
# === Run training ===
train_model(epochs=10)

Epoch 1 | Train Loss: 0.7220 | Train Acc: 46.88% | Val Loss: 0.7188 | Val Acc: 52.50%
Epoch 2 | Train Loss: 0.5832 | Train Acc: 80.62% | Val Loss: 0.5940 | Val Acc: 80.00%
Epoch 3 | Train Loss: 0.5243 | Train Acc: 82.50% | Val Loss: 0.5299 | Val Acc: 77.50%
Epoch 4 | Train Loss: 0.4310 | Train Acc: 88.75% | Val Loss: 0.4793 | Val Acc: 85.00%
Epoch 5 | Train Loss: 0.3971 | Train Acc: 89.38% | Val Loss: 0.4365 | Val Acc: 90.00%
Epoch 6 | Train Loss: 0.3757 | Train Acc: 90.00% | Val Loss: 0.3980 | Val Acc: 92.50%
Epoch 7 | Train Loss: 0.3282 | Train Acc: 94.38% | Val Loss: 0.3395 | Val Acc: 90.00%
Epoch 8 | Train Loss: 0.3270 | Train Acc: 91.25% | Val Loss: 0.3073 | Val Acc: 90.00%
Epoch 9 | Train Loss: 0.2740 | Train Acc: 95.62% | Val Loss: 0.2890 | Val Acc: 92.50%
Epoch 10 | Train Loss: 0.2742 | Train Acc: 93.75% | Val Loss: 0.2793 | Val Acc: 87.50%

🏆 Best Val Accuracy: 92.50%


Model Test

In [43]:
def predict_single_image(image_path, model, class_names):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        output = model(img_tensor)
        probs = F.softmax(output, dim=1)
        _, predicted = torch.max(probs, 1)

    print(f"Predicted Class: {class_names[predicted.item()]}")
    print(f"Class Probabilities: {probs.squeeze().numpy()}")

In [47]:
# Assuming dataset = ImageFolder(...)
class_names = dataset.classes  # ['healthy', 'infected']

# Path to one test image
test_image_path_1 = "processed_data/serie infected leaves/infected_05.png"

predict_single_image(test_image_path_1, model_effb0, class_names)

Predicted Class: series_infected_leaves_augmented
Class Probabilities: [0.16856721 0.83143276]


In [48]:
test_image_path_2 = "processed_data/serie healthy leaves/healthy_05.png"
predict_single_image(test_image_path_2, model_effb0, class_names)

Predicted Class: serie_healthy_leaves_augmented
Class Probabilities: [0.7026029  0.29739708]


Model Evaluation

In [52]:
from sklearn.metrics import classification_report

def evaluate_final_model():
    model_effb0.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_effb0(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("\n📊 Final Evaluation on Validation Set:")
    print(classification_report(all_labels, all_preds, target_names=val_dataset.classes, digits=2))

# Run this after training
evaluate_final_model()


📊 Final Evaluation on Validation Set:
                                  precision    recall  f1-score   support

  serie_healthy_leaves_augmented       0.89      0.85      0.87        20
series_infected_leaves_augmented       0.86      0.90      0.88        20

                        accuracy                           0.88        40
                       macro avg       0.88      0.88      0.87        40
                    weighted avg       0.88      0.88      0.87        40

